In [154]:
import numpy as np
import math
import matplotlib.pyplot as plt
from numpy import linalg as LA
from scipy.linalg import hilbert
from scipy import linalg as LA

### Teil a

In [ ]:
# def lu(A):
    

In [157]:

#Spalten nach Aufgabenstellung

#erste Spalte
erste=np.arange(2,14,1)
print(f'Erste Spalte: {erste}\n')

from numpy import linalg as LA

x=[]
zweite=[]
dritte=[]
vierte=[]
fuenfte=[]
sechste=[]

for i in erste:
    
    A=hilbert(i)
    m,n=A.shape
    x=np.zeros(m)
    x[0]=1
    x[n-1]=1
    b = A[:,0] + A[:,n-1]
    
    #2.Spalte: Inf-Norm
    konditionszahl=LA.cond(A, np.inf)
    zweite.append(konditionszahl)
    
    
    #3. Spalte: Relative Fehler mit xbs=A\b
    
    xbs=np.asarray(LA.solve(A,b))
    dritte.append(LA.norm(x-xbs, np.inf)/LA.norm(x, np.inf))  
    
    #4. Spalte: Relative Fehler (xlr) mit [L,R,P]=lu(A) - LR-Zerlegung mit Permutationsmatrix
    
    # xlr=np.asarray(lu(A))
    # vierte=np.append(vierte, LA.norm(x-xlr, np.inf)/LA.norm(x, np.inf))
    
    #5. Spalte: Relative Fehler (xchol) R=chol(A) mit Cholensky-Zerlegung
    
    # xchol=np.asarray(chol(A))
    # fuenfte=np.append(fuenfte, LA.norm(x-xchol, np.inf)/LA.norm(x, np.inf))
    
    #6. Spalte: ||Axchol-b||_inf
    
    # sechste.append
    

#Ausgabe der Daten     
zweite=np.asarray(np.round(zweite), dtype=np.float64)
zweite=zweite*10e-16
print(f'Zweite Spalte: {zweite} \n') #### mit 10e-16 multiplizieren???? #####
print(f'Dritte Spalte: {dritte} \n')
print(f'Vierte Spalte: {vierte} \n')
print(f'Fünfte Spalte: {fuenfte} \n')
print(f'Sechste Spalte: {sechste} \n')






    
    

Erste Spalte: [ 2  3  4  5  6  7  8  9 10 11 12 13]

Zweite Spalte: [2.70000000e-14 7.48000000e-13 2.83750000e-11 9.43656000e-10
 2.90702790e-08 9.85194887e-07 3.38727891e-05 1.09964990e-03
 3.53529487e-02 1.23050361e+00 3.92020098e+01 8.53495393e+02] 

Dritte Spalte: [6.661338147750939e-16, 3.330669073875469e-16, 3.26905169600877e-13, 3.652502875767687e-12, 1.7722270866306766e-10, 2.9427053497843923e-09, 4.390994877811886e-08, 1.6418006026947482e-06, 4.9167607710617043e-05, 0.0003068096178918075, 0.0450606125510078, 1.8259548699052652] 

Vierte Spalte: [] 

Fünfte Spalte: [] 

Sechste Spalte: [] 



In [216]:
def lu(A):
    m,n=A.shape

    L = [[0 for x in range(n)]
             for y in range(n)]
    R = [[0 for x in range(n)]
             for y in range(n)]
    P=np.identity(n).tolist()
    
    p_index=np.arange(0,n,1)
    print(p_index)
    
    
    # lR-Zerlegung
    for i in range(n):
 
        # R(obere Dreiecksmatrix)
        for k in range(i, n):
 
            # Summe von L(i, j) * R(j, k)
            sum = 0
            for j in range(i):
                sum += (L[i][j] * R[j][k])
 
            # Algorithmus R(i, k)
            R[i][k] = A[i][k] - sum
 
        # L(untere Dreiecksmatrix)
        for k in range(i, n):
            if (i == k):
                L[i][i] = 1  # 1 in der Diagonal
            else:
                # Summe von L(k, j) * U(j, i)
                sum = 0
                for j in range(i):
                    sum += (L[k][j] * R[j][i])
                    
 
                # Algorithmus L(k, i)
                L[k][i] = int((A[k][i] - sum) / R[i][i])
 
    # Ausgabe der Matrizen :
    print("Untere Dreiecksmatrix\t\tObere Dreiecksmatrix")
    
    for i in range(n):
 
        # L-Matrix
        for j in range(n):
            print(L[i][j], end="\t")
        print("", end="\t")
 
        # R-Matrix
        for j in range(n):
            print(R[i][j], end="\t")
        print("")

In [217]:
A=np.array(([2,-3,4,3],[6,-9,12,8],[4,-5,6,7],[2,1,-1,8]))
A.shape
B=hilbert(4)


lu(B)

[0 1 2 3]
Untere Dreiecksmatrix		Obere Dreiecksmatrix
1	0	0	0		1.0	0.5	0.3333333333333333	0.25	
0	1	0	0		0	0.3333333333333333	0.25	0.2	
0	0	1	0		0	0	0.2	0.16666666666666666	
0	0	0	1		0	0	0	0.14285714285714285	
